In [1]:
from pyspark import SparkContext, SparkConf
from operator import add

conf = SparkConf().setAppName("Movies")
sc = SparkContext(conf = conf) 

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

from pyspark.sql.types import *

In [2]:
df_users = sc.textFile('./moviedata/users.csv')
df_zipcode = sc.textFile('./moviedata/zipcodes.csv')
df_rating = sc.textFile('./moviedata/rating.csv')
# df = sqlContext.read.format('com.databricks.spark.csv').load('./moviedata/rating.csv')

In [12]:
user_zipcode = df_users.map(lambda x: x.split(",")).map(lambda x:  (str(x[4]), 1))

In [19]:
zipcode_usercount= user_zipcode.reduceByKey(lambda x, y: x+y)
zipcode_usercount.take(5)

[('"77845"', 2),
 ('"60090"', 1),
 ('"76059"', 1),
 ('"29206"', 1),
 ('"92037"', 1)]

In [20]:
city_zipcode = df_zipcode.map(lambda x: x.split(",")).map(lambda x: (str(x[0]), str(x[2])))
city_zipcode.take(4)

[('"2574"', '"WEST FALMOUTH"'),
 ('"1886"', '"WESTFORD"'),
 ('"1472"', '"WEST GROTON"'),
 ('"2671"', '"WEST HARWICH"')]

In [21]:
f = zipcode_usercount + city_zipcode

In [22]:
def city_usercount_join(x, y):
    print(type(x))
    if type(x) == int:
        return (y , x)
    return (x, y)

In [23]:
g = f.reduceByKey(lambda x, y: (x, y)).filter(lambda x : type(x[1])!=str).map(lambda x: x[1])
g.collect()

[(1, '"RANGELY"'),
 (1, '"SILVER SPRING"'),
 (1, '"BROOKLYN"'),
 (1, '"RIVERSIDE"'),
 (1, '"MILWAUKEE"'),
 (1, '"SAN ANTONIO"'),
 (1, '"AFTON"'),
 (1, '"RICHMOND"'),
 (1, '"DENVER"'),
 (1, '"PUTNAM"'),
 (1, '"BRIGHAM CITY"'),
 (1, '"EL CAJON"'),
 (1, '"BUFFALO GAP"'),
 (1, '"WHEELING"'),
 (1, '"COLUMBIA"'),
 (1, '"OVERLAND PARK"'),
 (1, '"RESEDA"'),
 (1, '"SACRAMENTO"'),
 (1, '"SAINT LEONARD"'),
 (1, '"CHARLOTTESVILLE"'),
 (1, '"ESSEX JUNCTION"'),
 (1, '"FLORISSANT"'),
 (1, '"JENISON"'),
 (1, '"BUFFALO"'),
 (1, '"RENO"'),
 (1, '"TALLAHASSEE"'),
 (1, '"CARMEL"'),
 (1, '"DUBLIN"'),
 (1, '"ITHACA"'),
 (2, '"FORT COLLINS"'),
 (1, '"PHILADELPHIA"'),
 (1, '"BATON ROUGE"'),
 (1, '"FORT LAUDERDALE"'),
 (1, '"ELK GROVE VILLAGE"'),
 (1, '"REDLANDS"'),
 (1, '"WAUKESHA"'),
 (1, '"TUCKAHOE"'),
 (1, '"DECATUR"'),
 (2, '"AUSTIN"'),
 (1, '"POWAY"'),
 (1, '"DECATUR"'),
 (1, '"ROLLINSFORD"'),
 (1, '"MONMOUTH"'),
 (1, '"LYNNFIELD"'),
 (1, '"SANTA CLARA"'),
 (1, '"DOWNERS GROVE"'),
 (4, '"MINNEAPOLIS"'),
